# Lab 9a. CohereAI Embeddings with LlamaIndex

Install LlamaIndex.

In [ ]:
%pip install -q llama-index-llms-litellm
%pip install -q llama-index-embeddings-cohere

In [ ]:
!pip install -qU llama-index

In [ ]:
import os

cohere_api_key = "tRGKPmSqy0QKq2f2LbB120INzMd1wYbP9s896Dl3"
os.environ["COHERE_API_KEY"] = cohere_api_key

#### With latest `embed-english-v3.0` embeddings.

- input_type="search_document": Use this for texts (documents) you want to store in your vector database

- input_type="search_query": Use this for search queries to find the most relevant documents in your vector database

In [ ]:
from llama_index.embeddings.cohere import CohereEmbedding

# with input_typ='search_query'
embed_model = CohereEmbedding(
    cohere_api_key=cohere_api_key,
    model_name="embed-english-v3.0",
    input_type="search_query",
)

embeddings = embed_model.get_text_embedding("Hello Team Oracle!")

print(len(embeddings))
print(embeddings[:5])

In [ ]:
# with input_type = 'search_document'
embed_model = CohereEmbedding(
    cohere_api_key=cohere_api_key,
    model_name="embed-english-v3.0",
    input_type="search_document",
)

embeddings = embed_model.get_text_embedding("Hello Team Oracle!")

print(len(embeddings))
print(embeddings[:5])

### let's use 
1. input_type=`search_document` to build index
2. input_type=`search_query` to retrive relevant context.

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

from llama_index.llms.litellm import LiteLLM
from llama_index.core.response.notebook_utils import display_source_node

from IPython.display import Markdown, display

#### Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

#### Load Data

In [ ]:
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

#### Build index with input_type = 'search_document'

In [ ]:
llm = LiteLLM("command-nightly")
embed_model = CohereEmbedding(
    cohere_api_key=cohere_api_key,
    model_name="embed-english-v3.0",
    input_type="search_document",
)

index = VectorStoreIndex.from_documents(
    documents=documents, embed_model=embed_model
)

#### Build retriever with input_type = 'search_query'

In [ ]:
embed_model = CohereEmbedding(
    cohere_api_key=cohere_api_key,
    model_name="embed-english-v3.0",
    input_type="search_query",
)

search_query_retriever = index.as_retriever()

search_query_retrieved_nodes = search_query_retriever.retrieve(
    "What happened in the summer of 1995?"
)

In [ ]:
for n in search_query_retrieved_nodes:
    display_source_node(n, source_length=2000)